In [1]:
import pandas as pd

/tmp/ipykernel_5253/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [51]:
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
animals = pd.read_csv("project1.csv")
animals.head()

,Animal ID,Date of Birth,Name,DateTime,MonthYear,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A680855,5/25/14,NaN,2014-06-10T00:00:00-05:00,Jun-14,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
1,A680857,5/25/14,NaN,2014-06-10T00:00:00-05:00,Jun-14,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
2,A680858,5/25/14,NaN,2014-06-10T00:00:00-05:00,Jun-14,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
3,A680859,5/25/14,NaN,2014-06-10T00:00:00-05:00,Jun-14,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
4,A680860,5/25/14,NaN,2014-06-10T00:00:00-05:00,Jun-14,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black


In [19]:
animals.shape

(131165, 12)

In [20]:
animals.size

1573980

In [45]:
animals.info()
animals.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131165 entries, 0 to 131164
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         131165 non-null  object
 1   Date of Birth     131165 non-null  object
 2   Name              93658 non-null   object
 3   DateTime          131165 non-null  object
 4   MonthYear         131165 non-null  object
 5   Outcome Type      131125 non-null  object
 6   Outcome Subtype   65810 non-null   object
 7   Animal Type       131165 non-null  object
 8   Sex upon Outcome  131165 non-null  object
 9   Age upon Outcome  131165 non-null  object
 10  Breed             131165 non-null  object
 11  Color             131165 non-null  object
dtypes: object(12)
memory usage: 12.0+ MB


17

In [46]:
animals.isnull().sum()

Animal ID               0
Date of Birth           0
Name                37507
DateTime                0
MonthYear               0
Outcome Type           40
Outcome Subtype     65355
Animal Type             0
Sex upon Outcome        0
Age upon Outcome        0
Breed                   0
Color                   0
dtype: int64

In [61]:
#All of the data types are objects and likely need to be converted to numerical values for valuable ML results. 
#There are also 17 duplicates among the rows and thus need to be treated. I would convery Date of Birth, DateTime, MonthYear into datetime format. Animal ID doesnt really need to be changed and can be eliminated as all IDs are unique.
#Outcome Type, Outcome Subtype, Animal Type, Sex upon Outcome, Breed, and Color can be converted to categorial using One-hot encoding. Lastly, Age Upon Outcome can be stripped to only the numerical value. 
animals.drop_duplicates(inplace=True, ignore_index=True)
animals.duplicated().sum()
#Lastly the Animal ID column, the Name column should be eliminated as they do not add any value to the analysis as each value are unique or don't affect the analysis numerically.
animals.drop(["Animal ID"], axis = 1, inplace=True)
animals.drop(["Name"], axis = 1, inplace=True)

In [62]:
animals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131148 entries, 0 to 131147
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Date of Birth     131148 non-null  object
 1   DateTime          131148 non-null  object
 2   MonthYear         131148 non-null  object
 3   Outcome Type      131108 non-null  object
 4   Outcome Subtype   65802 non-null   object
 5   Animal Type       131148 non-null  object
 6   Sex upon Outcome  131148 non-null  object
 7   Age upon Outcome  131148 non-null  object
 8   Breed             131148 non-null  object
 9   Color             131148 non-null  object
dtypes: object(10)
memory usage: 10.0+ MB


In [ ]:
#Now after removing duplicated rows and useless columns the data types of eac